<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):  #input_dim:features
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)
        print('outputs.shape = ',outputs.shape)
        print('hidden.shape = ',hidden.shape)
        print('cell.shape = ',cell.shape)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs,hidden, cell

In [57]:
model = Encoder(15,16,1)
model

Encoder(
  (rnn): LSTM(15, 16, batch_first=True)
)

In [58]:
input_data = torch.rand(1, 10, 15)
output,hidden, cell = model(input_data)

outputs.shape =  torch.Size([1, 10, 16])
hidden.shape =  torch.Size([1, 1, 16])
cell.shape =  torch.Size([1, 1, 16])


In [61]:
output[:,-1,:]== hidden

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True]]])

In [64]:
hidden ==cell

tensor([[[False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False]]])

In [97]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers): #embedding dimensions
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=True,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)
        print('outputs.shape = ',outputs.shape)
        print('hidden.shape = ',hidden.shape)
        print('cell.shape = ',cell.shape)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs[:,-1:,:]  #outputs[:,-1:,:]=output of the end of blocks which is concatenated of two directions(we use -1: for keeping the dimensions)

In [102]:
model = Encoder(15,32,1) #embedding dimensions
model

Encoder(
  (rnn): LSTM(15, 32, batch_first=True, bidirectional=True)
)

In [103]:
input_data = torch.rand(1, 10, 15)
output= model(input_data)
print(output.shape)

outputs.shape =  torch.Size([1, 10, 64])
hidden.shape =  torch.Size([2, 1, 32])
cell.shape =  torch.Size([2, 1, 32])
torch.Size([1, 1, 64])


In [91]:
input_data = torch.rand(1, 10, 15)
print(input_data[:,-1,:].shape)
print(input_data[:,-1:,:].shape)

torch.Size([1, 15])
torch.Size([1, 1, 15])


In [81]:
import numpy as np

# Create a vector of shape [1, 5]
vector = np.array([[1, 2, 3, 4, 5]])

# Repeat the vector along the second dimension to create a tensor of shape [1, 10, 5]
tensor = np.tile(vector, (1, 10, 1))

# Check the shape of the resulting tensor
print(tensor.shape)
print(tensor)


(1, 10, 5)
[[[1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]]]


In [85]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):       #input _dim depends on encoder's output
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=False,batch_first=True)

    def forward(self, src):

        #src = [src len, batch size]
        src = torch.tile(src, (1, 10, 1)) # give the output of encoder to each cell of decoder

        outputs, (hidden, cell) = self.rnn(src)
        print('outputs.shape = ',outputs.shape)
        print('hidden.shape = ',hidden.shape)
        print('cell.shape = ',cell.shape)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs

In [106]:
model = Decoder(64,15,1)
model

Decoder(
  (rnn): LSTM(64, 15, batch_first=True)
)

In [107]:
input_data = torch.rand(1, 1, 64)
output= model(input_data)
print(output.shape)

outputs.shape =  torch.Size([1, 10, 15])
hidden.shape =  torch.Size([1, 1, 15])
cell.shape =  torch.Size([1, 1, 15])
torch.Size([1, 10, 15])


In [127]:
class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):
        encoded = self.encoder(src)
        decoded = self.decoder(encoded)
        return decoded

In [128]:
encoder = Encoder(15,32,1)
decoder = Decoder(64,15,1)
model = Autoencoder(encoder, decoder)
model

Autoencoder(
  (encoder): Encoder(
    (rnn): LSTM(15, 32, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(64, 15, batch_first=True)
  )
)

In [129]:
input_data = torch.rand(1, 10, 15)
output= model(input_data)
print(output.shape)

outputs.shape =  torch.Size([1, 10, 64])
hidden.shape =  torch.Size([2, 1, 32])
cell.shape =  torch.Size([2, 1, 32])
outputs.shape =  torch.Size([1, 10, 15])
hidden.shape =  torch.Size([1, 1, 15])
cell.shape =  torch.Size([1, 1, 15])
torch.Size([1, 10, 15])
